In [436]:
import pandas as pd
import numpy as np
import folium as fm
import re

In [437]:
import os
os.path.exists('./Data/P3_GrantExport.csv')
data = pd.read_csv('./Data/P3_GrantExport.csv', delimiter=';', na_values=['.'], error_bad_lines=False)

We drop all unuseful columns

In [438]:
data.drop('Project Title', axis=1, inplace=True)
data.drop('Project Title English', axis=1, inplace=True)
data.drop('Responsible Applicant', axis=1, inplace=True)
data.drop('Funding Instrument', axis=1, inplace=True)
data.drop('Funding Instrument Hierarchy', axis=1, inplace=True)
data.drop('Institution', axis=1, inplace=True)
data.drop('Discipline Number', axis=1, inplace=True)
data.drop('Discipline Name', axis=1, inplace=True)
data.drop('Discipline Name Hierarchy', axis=1, inplace=True)
data.drop('Keywords', axis=1, inplace=True)

We drop project with missing starting date or ending date

In [439]:
len(data)

63969

In [440]:
data = data[pd.notnull(data['Start Date'])]
data = data[pd.notnull(data['End Date'])]

In [441]:
len(data)

63968

We drop the "Nicht zuteilbar" in the university column

In [442]:
data = data[data.University != 'Nicht zuteilbar - NA']

In [443]:
data['Start Date'] = pd.to_datetime(data['Start Date'])
data['End Date'] = pd.to_datetime(data['End Date'])

In [444]:
sortStartDate  = data.sort_values(by='Start Date')

In [445]:
sortStartDate.drop_duplicates('University', keep='first')

,"﻿""Project Number""",University,Start Date,End Date,Approved Amount
1854,2483,Université de Fribourg - FR,1975-01-06,1976-09-30,247904.00
1937,2575,ETH Zürich - ETHZ,1975-01-07,1975-12-31,8000.00
31,41,"NPO (Biblioth., Museen, Verwalt.) - NPO",1975-01-08,1976-10-31,8902.00
64,83,Universität Bern - BE,1975-01-08,1976-09-30,14235.00
63,82,Université de Neuchâtel - NE,1975-01-08,1976-01-31,13620.00
1890,2521,EPF Lausanne - EPFL,1975-01-10,1976-09-30,69539.00
1896,2528,Universität Zürich - ZH,1975-01-10,1977-06-30,214663.00
1898,2531,Universität Basel - BS,1975-01-10,1976-09-30,106124.00
1903,2536,Université de Lausanne - LA,1975-01-10,1976-09-30,53465.00
1904,2539,Université de Genève - GE,1975-01-10,1976-09-30,1559894.00


We can see above the lastest project for each university. We decided to take into account only data after the 08-01-2012 since this is the first project that we have for the more recent university.

In [446]:
min = []
for group in groupByUni:
    print(group.min())

NameError: name 'groupByUni' is not defined

In [450]:
universities = data['University'].unique()

university_names = universities.copy()

In [451]:
ids = {}
for record in university_names:
    wds = str(record).split()
    name = ' '.join(wds[:-2])
    if name:
        ids[name] = wds[-1]
ids

{'AO Research Institute': 'AORI',
 'Allergie- und Asthmaforschung': 'SIAF',
 'Berner Fachhochschule': 'BFH',
 'Biotechnologie Institut Thurgau': 'BITG',
 "Centre de rech. sur l'environnement alpin": 'CREALP',
 'EPF Lausanne': 'EPFL',
 'ETH Zürich': 'ETHZ',
 'Eidg. Anstalt für Wasserversorgung': 'EAWAG',
 'Eidg. Forschungsanstalt für Wald,Schnee,Land': 'WSL',
 'Eidg. Hochschulinstitut für Berufsbildung': 'EHB',
 'Eidg. Material und Prüfungsanstalt': 'EMPA',
 'Ente Ospedaliero Cantonale': 'EOC',
 'Fachhochschule Kalaidos': 'FHKD',
 'Fachhochschule Nordwestschweiz (ohne PH)': 'FHNW',
 'Fachhochschule Ostschweiz': 'FHO',
 'Facoltà di Teologia di Lugano': 'FTL',
 'Fernfachhochschule Schweiz (Mitglied SUPSI)': 'FFHS',
 'Firmen/Privatwirtschaft': 'FP',
 'Forschungsanstalten Agroscope': 'AGS',
 'Forschungsinstitut für Opthalmologie': 'IRO',
 'Forschungsinstitut für biologischen Landbau': 'FIBL',
 'Franklin University Switzerland': 'FUS',
 'Friedrich Miescher Institute': 'FMI',
 'HES de Suisse 

In [ ]:
cantons_id = ['ZH', 'BE','LU', 'UR', 'SZ', 'OW','NW', 'GL', 'ZG', 'FR',
                          'SO','BS', 'BL', 'SH', 'AR', 
                          'AI', 'SG', 'GR', 'AG', 'TG', 'TI', 'VD',
                          'VS', 'NE', 'GE','JU']

In [455]:
universities_pd = pd.DataFrame({'full university name':university_names})
university_names = universities_pd['full university name'].tolist()
name_list = []
abbrev_list = []
# row_list = []

for i, uni in enumerate(university_names):
    split_list = str(uni).split(' - ')
    if(len(split_list) == 1):
        split_list.append('')
    name_list.append(split_list[0])
    abbrev_list.append(split_list[1])
    
universities_pd['university name'] = name_list
universities_pd['university abbrev'] = abbrev_list
# universities_pd['canton'] = universities_pd['university abbrev'] if universities_pd['university abbrev'] in cantons_id

In [456]:
import simplejson as json
DOMAIN = 'http://api.geonames.org/'#api.geonames.org/search?
USERNAME = 'shynkaru' #geonames username

def geonames_query(method, params):
    uri = DOMAIN + '%s?%s&username=%s' % (method, urllib.parse.urlencode(params), USERNAME)
    resource = urllib.request.urlopen(uri).readlines()
    return json.loads(resource[0])

In [457]:
def prepare_params(university, format_type):
    return { 'q' : university
            , 'country' : "ch"
            , 'lang': 'en'
            , 'featurecode':'univ'
            , 'type':format_type}

In [458]:
def try_query(university):
    if not university:
        return None
    
    params = prepare_params(university, 'json')
    response_json = geonames_query('search', params)
    
    if(not response_json['geonames']):
        return try_query(' '.join(university.split()[:-1]))
    else:
        return response_json

In [459]:
def query_university(university):
    response_json = try_query(university)
    if(not response_json):
        return None
   
    out_json = response_json['geonames']
    total_results = int(response_json['totalResultsCount'])
    
    l = [( out_json[i]['name']
          , out_json[i].get('adminCode1', 'Missing: adminCode1')
          , out_json[i]['adminName1']
          , out_json[i]['lat']
          , out_json[i]['lng']) for i in range(len(out_json))]
        
    return l

In [460]:
def process_data():
    for university, canton in ids.items():
        print(university)
        possible_uni_data = query_university(university)
        if not possible_uni_data:
            print(None, '\n')
            continue
            
        for item in possible_uni_data:
            print(item)
        print('\n')

process_data()

Friedrich Miescher Institute
('Basel, Friedrich-Miescher-Str.', 'BS', 'Basel-City', '47.57389', '7.56484')


Universität Zürich
('Völkerkundemuseum der Universität Zürich', 'ZH', 'Zurich', '47.37092', '8.53434')


Eidg. Forschungsanstalt für Wald,Schnee,Land
('Eidg. Forschungsanstalt', 'ZH', 'Zurich', '47.36221', '8.4545')


Université de Neuchâtel
('Neuchâtel, Université', 'NE', 'Neuchâtel', '46.99385', '6.93789')


Allergie- und Asthmaforschung
None 

NPO (Biblioth., Museen, Verwalt.)
None 

Fachhochschule Nordwestschweiz (ohne PH)
('FHNW University of Applied Sciences Northwestern Switzerland, Muttenz', 'BL', 'Basel-Landschaft', '47.53378', '7.63856')


Istituto Svizzero di Roma
None 

Weitere Spitäler
None 

EPF Lausanne
None 

SUP della Svizzera italiana
('Lac Sup. de Fully', 'VS', 'Valais', '46.17852', '7.09391')


Centre de rech. sur l'environnement alpin
('CERN', 'GE', 'Geneva', '46.23333', '6.05')
('Echallens', 'VD', 'Vaud', '46.63845', '6.63525')
('Renens (VD)', 'VD', 'Vaud',